In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

font = {'size':20,
        'family':'serif',
        'style':'normal'}
plt.rc('font', **font)

In [ ]:
df = pd.read_csv('/kaggle/input/500-richest-people-2021/500 richest people 2021.csv', delimiter = ';')

In [ ]:
df = df.iloc[:, 0:7]
df.head()

In [ ]:
df.shape

In [ ]:
df[df.isna().any(1)]

In [ ]:
indices = df[df.isna().any(1)].index
df.drop(indices, inplace = True)
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
def manipulate(df, col):
    df[col] = df[col].replace({'\$':''}, regex =  True)

def to_billion(x):
    if 'M' in x:
        return float(x.rstrip('M')) / 1000
    elif 'k' in x:
        return float(x.rstrip('k')) / 10 ** 6
    else:
        return float(x.rstrip('B'))
    
manipulate(df, '$ Last Change')
#manipulate(df, 'Total Net Worth')

df['$ Last Change'] = df['$ Last Change'].apply(to_billion)
#df['Total Net Worth'] = df['Total Net Worth'].replace({'\$':'', 'B':''}, regex = True).astype(float)

In [ ]:
ind = df[df.Country == 'India']

In [ ]:
plt.figure(figsize = (15, 8))

plt.bar(ind['Name'], ind['$ Last Change'].astype(float), color = (ind['$ Last Change'] > 0.0).map({True:'green',
                                                                               False:'maroon'}))
plt.xticks(rotation = 90)

In [ ]:
ind['Total Net Worth'] = ind['Total Net Worth'].replace({'\$':'', 'B':''}, regex = True).astype(float)

plt.figure(figsize = (15, 8))

plt.bar(ind['Name'], ind['Total Net Worth'].astype(float), color = (ind['Name'] == 'Mukesh Ambani').map({True:'maroon',
                                                                                                         False:'blue'}))
plt.xticks(rotation = 90)
plt.title('Wealth (in billion $) of top Indian Businessmen')

In [ ]:
def AvgChange():
    d = {}
    for i in zip(df['Industry'].unique()):
        median = df[df['Industry'] == i[0]]['$ Last Change'].median()
        d[i[0]] = median
    return pd.Series(d).sort_values()

In [ ]:
for i in zip(df['Industry'].unique()):
    print(i[0])

In [ ]:
df['Industry'].unique()

In [ ]:
AvgChange()

In [ ]:
plt.figure(figsize = (15, 8))

plt.bar(AvgChange().index, AvgChange().values, color = np.where(AvgChange().index == 'Retail', 'maroon', 'firebrick'))
plt.ylim([-0.20, 0.05])
plt.xticks(rotation = 90)

In [ ]:
df[df['Total Net Worth'] == df['Total Net Worth'].iloc[84]]

In [ ]:
#df.at(84, 'Total Net Worth') = 18.7

df['Total Net Worth'] = df['Total Net Worth'].replace({'\$':'', 'B':'', 'family':'18.7'}, regex = True)
df['Total Net Worth'] = pd.to_numeric(df['Total Net Worth'])

In [ ]:
df[df['Total Net Worth'] == df['Total Net Worth'].iloc[84]]

In [ ]:
plt.figure(figsize = (15, 8))

plt.bar(ind['Name'], ind['Total Net Worth'].astype(float), color = (ind['Name'] == 'Mukesh Ambani').map({True:'maroon',
                                                                                                         False:'blue'}))
plt.axhline(df['Total Net Worth'].mean(), color = 'black', ls = ':', lw = 10, label = 'Mean total net worth (world)')
plt.axhline(ind['Total Net Worth'].mean(), color = 'forestgreen', ls = '--', lw = 10, label = 'Mean total net worth (India)')
plt.xticks(rotation = 90)
plt.title('Wealth (in billion $) of top Indian Businessmen')
plt.legend(loc = 1, shadow = True, edgecolor = 'black')

In [ ]:
df['Total Net Worth'].mean()

In [ ]:
df.head()

In [ ]:
plt.figure(figsize = (15, 8))

plt.barh(df['Industry'].value_counts().index, df['Industry'].value_counts().values, color = '#3290a8')
plt.xticks(rotation = 90)

In [ ]:
plt.figure(figsize = (15, 8))

plt.barh(ind['Industry'].value_counts().index, ind['Industry'].value_counts().values, color = '#3290a8')
plt.xticks(rotation = 90)

In [ ]:
plt.figure(figsize = (15, 8))

plt.bar(df['Country'].value_counts()[:10].index, df['Country'].value_counts()[:10].values, color = '#3290a8')
plt.xticks(rotation = 90)

In [ ]:
import math

def EvalCdf(sample, x):
    count = 0
    for i in sample:
        if i <= x:
            count += 1
    return count / len(sample)

def PercentileToRank(sample, perc):
    pos = (perc / 100) * len(sample)
    val = sorted(sample)[math.ceil(pos - 1)]
    return val

In [ ]:
df.head()

In [ ]:
total_worth = sorted(df['Total Net Worth'].values)
cdf_total_worth = [EvalCdf(total_worth, x) for x in total_worth]

plt.figure(figsize = (15, 8))

plt.plot(total_worth, cdf_total_worth, color = '#a87332', label = 'CDF of the total wealth accumulated by the top 500 Billionaires')
#plt.axhline(0.9, alpha = 0.9, color = 'grey')
plt.hlines(0.9, xmin = PercentileToRank(total_worth, 90), xmax = PercentileToRank(total_worth, 100), color = 'black', ls = ':')
plt.axvline(PercentileToRank(total_worth, 90), alpha = 0.9, color = 'black', ls = ':')
plt.axvline(PercentileToRank(total_worth, 100), alpha = 0.9, color = 'black', ls = ':')

#plt.fill_betweenx(y = [0.9], x1 = [PercentileToRank(total_worth, 90)], x2 = [PercentileToRank(total_worth, 100)])
plt.axvspan(PercentileToRank(total_worth, 90), PercentileToRank(total_worth, 100), ymin = 0.04, ymax = 0.86, color = 'grey',
            alpha = 0.3, in_layout = True, label = 'Wealth accrued by the top 50 Billionaires')

plt.legend(loc = 7, shadow = True, edgecolor = 'black')

In [ ]:
PercentileToRank(total_worth, 90)

In [ ]:
int((80 / 100) * 500)

In [ ]:
math.ceil(3.96)

In [ ]:
df[df['Total Net Worth'] == 29.8]